In [ ]:
import numpy as np

In [ ]:
# Environment and Policy Setup
# State space: S0 (Start), S_A (Target), S_B, S_C, S_D (Terminal)
# Rewards are obtained upon transitioning to the next state.
STATES = ['S0', 'S_A', 'S_B', 'S_C']
TERMINAL_STATE = 'S_D'

In [ ]:
TRANSITIONS = [
    ('S0', 'S_A', 0),
    ('S_A', 'S_B', 1),  # Visit 1 to S_A: Immediate Reward 1
    ('S_B', 'S_C', 0),
    ('S_C', 'S_A', 0),  # Loop back to S_A: Reward 0
    ('S_A', 'S_D', 10), # Visit 2 to S_A: Terminal Reward 10 (High reward for reaching the end)
]

In [ ]:
# Function to generate one episode 
def generate_episode():
    episode = []
    # State, Reward pair for the sequence
    episode.append(('S0', 0))
    episode.append(('S_A', 1)) # R1 (from S_A to S_B)
    episode.append(('S_B', 0)) # R2 (from S_B to S_C)
    episode.append(('S_C', 0)) # R3 (from S_C to S_A)
    episode.append(('S_A', 10))# R4 (from S_A to S_D)
    # The last state (S_D) doesn't yield a reward, so we stop here.
    return episode

In [ ]:
# Monte Carlo Estimation Functions

def estimate_v_mc(num_episodes, method='first-visit', target_state='S_A', gamma=1.0):
    
    # Store all returns observed for the target state
    returns_list = []
    
    for _ in range(num_episodes):
        episode = generate_episode()
        
        # Calculate Returns(G_t)for all steps
        G = {}
        cumulative_return = 0
        
        # Iterate backward to calculate the discounted return G_t
        for t in range(len(episode) - 1, -1, -1):
            state = episode[t][0]
            reward = episode[t][1]

            cumulative_return = reward + gamma * cumulative_return
            G[t] = cumulative_return

        # Collect Returns for the Target State ('S_A')
        
        # Keep track of the first visit index if needed
        first_visit_index = -1
        
        # Iterate forward to collect the returns
        for t in range(len(episode)):
            state = episode[t][0]
            
            if state == target_state:
                if method == 'first-visit':
                    # Only consider the return from the first time S_A is encountered
                    if first_visit_index == -1:
                        first_visit_index = t
                        returns_list.append(G[t])
                        # Stop collecting for this episode once the first visit is processed
                        break 
                
                elif method == 'every-visit':
                    # Consider the return from every time S_A is encountered
                    returns_list.append(G[t])
    
    #  Calculate the Estimated Value
    if returns_list:
        estimated_value = np.mean(returns_list)
    else:
        estimated_value = 0
        
    return estimated_value, len(returns_list)